#Text clustering -sample code


In [0]:
import numpy as np
import pandas as pd 
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
#load dataset into pandas dataframe
df_news = pd.read_csv('https://raw.githubusercontent.com/ytwang3579/TextMining/master/dataset_0606.csv',encoding="utf-8")
data=df_news[df_news['source']=='focustaiwan']

In [0]:
doc_data=data.drop_duplicates(subset=['title'], keep=False)

In [3]:
# load data to pandas data frame
import pandas as pd
import numpy as np

news_titles = doc_data['title'].tolist()  # convert column Title of panda frame to list
news_content = doc_data['content'].tolist()
doc_data

,date,title,content,url,source,section
1080,8/3/2020,Mets Taiwan Day 2020 to offer limited edition 'Taiwan No. 1' jersey,"New York, March 7 (CNA) The New York Mets organization will give away 500 Mets replica jerseys decorated with the name ""Taiwan"" and ""No. 1"" during the Major League Baseball (MLB) team's 16th annua...",https://focustaiwan.tw/sports/202003080006,focustaiwan,Sports
1081,9/3/2020,Taiwanese boxer secures ticket to Tokyo Olympics in Amman qualifier,"Taipei, March 9 (CNA) Taiwanese boxer Chen Nien-chin (陳念琴) on Sunday advanced to the semifinals of the Asia/Oceania Olympic Boxing Qualification event in Jordan, thereby earning a place in the 202...",https://focustaiwan.tw/sports/202003090007,focustaiwan,Sports
1082,9/3/2020,Second Taiwanese boxer secures ticket to Tokyo Olympics,"Taipei, March 9 (CNA) Taiwanese boxer Lin Yu-ting (林郁婷) beat 2016 Rio Olympics silver medalist Monday to advance to the semifinals of the Asia/Oceania Olympic Boxing Qualification event in Jordan,...",https://focustaiwan.tw/sports/202003090025,focustaiwan,Sports
1083,10/3/2020,Two more Taiwanese boxers qualify for Tokyo Olympics,"Taipei, March 10 (CNA) Taiwanese boxers Huang Hsiao-wen (黃筱雯) and Wu Shih-yi (吳詩儀) advanced to the semifinals of the Asia/Oceania Olympic Boxing Qualification event in Jordan on Monday, thereby ea...",https://focustaiwan.tw/sports/202003100010,focustaiwan,Sports
1084,14/3/2020,"Taiwan's Tai, Chou smash into semis at All England Badminton Open","Taipei, March 14 (CNA) Taiwanese badminton aces Tai Tzu-ying (戴資穎) and Chou Tien-chen (周天成) advanced into the semifinals of their respective categories at the Yonex All England Open after straight...",https://focustaiwan.tw/sports/202003140007,focustaiwan,Sports
...,...,...,...,...,...,...
2919,1/6/2020,Customers reminisce about bookstore where 'lights never went out',"Taipei, June 1 (CNA) A Taipei bookstore that had been open to customers around the clock since 1999 closed its doors on Sunday, but its special ambience and services will not be easily erased from...",https://focustaiwan.tw/culture/202006010013,focustaiwan,Culture
2920,2/6/2020,East Rift Valley Land Art Festival opens in Taitung,"Taipei, June 2 (CNA) The East Rift Valley Land Art Festival opened in Taitung County on Tuesday, featuring installations and other types of displays by local and foreign artists. The 22 artworks a...",https://focustaiwan.tw/culture/202006020022,focustaiwan,Culture
2921,3/6/2020,Documentary on Taiwanese food scores U.S. film prizes,"Taipei, June 3 (CNA) A short documentary focusing on Taiwanese food won two film awards in the United States recently, according to its producer, Deutsche Welle (DW). ""Tasty Taipei -- Food, Fusion...",https://focustaiwan.tw/culture/202006030012,focustaiwan,Culture
2922,4/6/2020,National Human Rights Museum wins award in Germany,"Taipei, June 4 (CNA) Taiwan's National Human Rights Museum (NHRM) won a ""Special Award"" at this year's Karl Wilhelm Fricke Award, for advancing education on human rights and democracy, the museum ...",https://focustaiwan.tw/culture/202006040013,focustaiwan,Culture


#Step 1:  Pre-Processing text

In [0]:
# Define function for Lemmatization, remove stopword and feature selection using POS, spacy package
import spacy
def spacy_preprocess (text,lemma= True, pos= True, pos_select = ["VERB", "NOUN", "ADJ","ADV","PART"]):
  # Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
  nlp = spacy.load('en', disable=['parser', 'ner']) # disable parser, ner for faster loading
  # Parse the sentence using the loaded 'en' model object `nlp`
  doc = nlp(text)
    
  if pos== False:
    if lemma== True: text_preprocess= " ".join([token.lemma_.lower() for token in doc if not nlp.vocab[token.text].is_stop])
    if lemma== False:text_preprocess= " ".join([token.text.lower() for token in doc if not nlp.vocab[token.text].is_stop])
  else:
    if lemma== True : text_preprocess= " ".join([token.lemma_.lower() for token in doc if (token.pos_ in pos_select and not nlp.vocab[token.text].is_stop)])
    if lemma== False : text_preprocess= " ".join([token.text.lower() for token in doc if (token.pos_ in pos_select  and not nlp.vocab[token.text].is_stop)])
  # nlp.vocab[token.text].is_stop to remove stopwords
  return text_preprocess

In [5]:
# Pre-processing data with spacy
from tqdm import tqdm
news_content_preprocess=[]
for news in tqdm(news_content):
  news_preprocess = spacy_preprocess(news,pos_select = ["VERB", "NOUN", "ADJ"])
  news_content_preprocess+= [news_preprocess]

100%|██████████| 1647/1647 [14:20<00:00,  1.91it/s]


#Step 2: Build feature matrix (tf-idf)
** you need to change the data input below** 

In [6]:
#without preprocessing
data_input =news_content # data without preprocess

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.8, max_features= None)
feature_matrix = vectorizer.fit_transform(data_input).astype(float)
feature_names = vectorizer.get_feature_names() # get feature names
print("number of feature:", len(feature_names))

number of feature: 333


#Step 3-1: Kmean Clustering

In [0]:
from sklearn.cluster import KMeans
#function for Kmean clustering
def k_means(feature_matrix, num_clusters=5):
    km = KMeans(n_clusters=num_clusters, n_init=500, random_state = 1,
                max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters

def get_cluster_data(clustering_obj, doc_data, 
                     feature_names, num_clusters,
                     topn_features=10):

    cluster_details = {}  
    # get cluster centroids
    ordered_centroids = clustering_obj.cluster_centers_.argsort()[:, ::-1]
    # get key features for each cluster
    # get docs belonging to each cluster
    for cluster_num in range(num_clusters):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster_num'] = cluster_num
        key_features = [feature_names[index] 
                        for index 
                        in ordered_centroids[cluster_num, :topn_features]]
        cluster_details[cluster_num]['key_features'] = key_features
        
        docs = doc_data[doc_data['Cluster'] == cluster_num]['section'].values.tolist()
        cluster_details[cluster_num]['docs'] = docs
    
    return cluster_details

def print_clusters(cluster_data):
    # print cluster details
    for cluster_num, cluster_details in cluster_data.items():
        print ('Cluster {} details:'.format(cluster_num))
        print ('-'*20)
        print ('Key features:', cluster_details['key_features'])
        print ('docs in this cluster:')
        print (', '.join(cluster_details['docs']))
        print ('='*40)


# labels has 7 classes
num_clusters = 7
km_obj, km_clusters = k_means(feature_matrix=feature_matrix,
                           num_clusters=num_clusters)

doc_data['Cluster'] = km_clusters

km_cluster_data =  get_cluster_data(clustering_obj=km_obj,
                                 doc_data = doc_data,
                                 feature_names=feature_names,
                                 num_clusters=num_clusters,
                                 topn_features=10)

print_clusters(km_cluster_data)  


In [0]:
def purity(km_cluster_data):
  all_items=[]
  total_samples=0
  for cluster_num, cluster_details in km_cluster_data.items():
    items=[]
    total_samples=total_samples+len(cluster_details["docs"])
    for i in cluster_details["docs"]:
      items.append(i[0]+i[1]+i[2]+i[3])
    all_items.append(items)

  sum=0
  from collections import Counter
  print("calculating purity score.")
  for items in all_items:
    newList=Counter(items).most_common(1)
    print("cluster:",newList)
    sum=sum+newList[0][1]
  #print(sum)
  #print(total_samples)
  print("purity score:",sum/total_samples)
  print ('*'*40)

purity(km_cluster_data)

In [0]:
# with pre-processing data
data_input =news_content_preprocess  # the data after preprocess text.

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.1, max_df=0.8, max_features= None)
feature_matrix = vectorizer.fit_transform(data_input).astype(float)
feature_names = vectorizer.get_feature_names() # get feature names
print("number of feature:", len(feature_names))

In [0]:
# labels has 7 classes
num_clusters = 7
km_obj, km_clusters = k_means(feature_matrix=feature_matrix,
                           num_clusters=num_clusters)

doc_data['Cluster'] = km_clusters

km_cluster_data =  get_cluster_data(clustering_obj=km_obj,
                                 doc_data = doc_data,
                                 feature_names=feature_names,
                                 num_clusters=num_clusters,
                                 topn_features=10)

print_clusters(km_cluster_data)  

In [0]:
purity(km_cluster_data)